In [1]:
import homemade_tensors as ht
from multilayer_perceptron import MultilayerPerceptron
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

C:\Users\samue\AppData\Local\Temp\ipykernel_24148\827387922.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
TEST_SIZE = 1
BATCH_SIZE = 64
EPOCHS = 20
LEARNING_RATE = 1e-3

## Loading data

You can download the mnist dataset from [here](https://www.kaggle.com/datasets/oddrationale/mnist-in-csv)

In [3]:
train_df = pd.read_csv("../data/mnist_train.csv")[:2000]  
test_df = pd.read_csv("../data/mnist_train.csv")[:1000]  

In [4]:
train_labels = np.eye(10)[train_df["label"].to_numpy()]
train_labels = [
    ht.Tensor(
        np.array(train_labels[i:i + BATCH_SIZE][:, np.newaxis, :])
    )
    for i in range(0, len(train_labels), BATCH_SIZE)
]
train_data = train_df.drop(["label"], axis=1).to_numpy() / 255
train_data = [
    ht.Tensor(
        np.array(train_data[i:i + BATCH_SIZE][:, np.newaxis, :])
    )
    for i in range(0, len(train_data), BATCH_SIZE)
]

test_labels = np.eye(10)[test_df["label"].to_numpy()]
test_labels = [
    ht.Tensor(
        np.array(test_labels[i:i + TEST_SIZE][:, np.newaxis, :])
    )
    for i in range(0, len(test_labels), TEST_SIZE)
]

test_data = test_df.drop(["label"], axis=1).to_numpy() / 255
test_data = [
    ht.Tensor(
        np.array(test_data[i:i + TEST_SIZE][:, np.newaxis, :])
    )
    for i in range(0, len(test_data), TEST_SIZE)
]

In [5]:
net = MultilayerPerceptron()

for epoch in range(EPOCHS):
    for label, image in zip(train_labels, train_data):
        result = net.forward(image)
        loss = ht.CrossEntropyLoss()(result, label)
        loss.backward()
        net.update_parameters(lr=LEARNING_RATE)
    print(f"Epoch {epoch + 1} Loss: {-1 / 10 * loss.data}")

Epoch 1 Loss: 3.686778340788532
Epoch 2 Loss: 3.6860088892000102
Epoch 3 Loss: 3.6795478276362616
Epoch 4 Loss: 3.6451499604765303
Epoch 5 Loss: 3.5698996756824224
Epoch 6 Loss: 3.474404205905988
Epoch 7 Loss: 3.3740996962555556
Epoch 8 Loss: 3.3046981219695746
Epoch 9 Loss: 3.2471690516537928
Epoch 10 Loss: 3.1873949510723465
Epoch 11 Loss: 3.129209555892995
Epoch 12 Loss: 3.0832120450431706
Epoch 13 Loss: 3.0430067072369194
Epoch 14 Loss: 3.004682949006284
Epoch 15 Loss: 2.9695227879534656
Epoch 16 Loss: 2.935334440004766
Epoch 17 Loss: 2.8998965625094684
Epoch 18 Loss: 2.865663038566637
Epoch 19 Loss: 2.8322130781960233
Epoch 20 Loss: 2.7934870321165137


In [6]:
correct = 0
for label, image in zip(test_labels, test_data):
    result = net.forward(image)
    prediciton = np.argmax(result.data, axis=-1)
    correct += np.count_nonzero(prediciton == np.argmax(label.data, axis=-1))

print(f"Accuracy: {correct / len(test_data) * 100}%")

Accuracy: 84.6%
